In [1]:
# =========================
# IMPORTS
# =========================

import boto3
import os
from boto3.s3.transfer import TransferConfig

print("🚀 Starting AWS S3 → RunPod S2 transfer (EfficientNet model)")

# =========================
# LOCAL FILE
# =========================

# Temporary local path on SageMaker
LOCAL_FILE_PATH = "/tmp/eff_ann_version8.h5"

# =========================
# SOURCE: AWS S3
# =========================

# Source bucket
SOURCE_BUCKET = "ai-bmi-predictor-v2"

# Source object key
SOURCE_KEY = (
    "trained-models/efficientnet-models/"
    "eff_ann_version8.h5"
)

print("📥 Downloading EfficientNet model from AWS S3")

# Use SageMaker IAM role
aws_s3 = boto3.client("s3")

aws_s3.download_file(
    Bucket=SOURCE_BUCKET,
    Key=SOURCE_KEY,
    Filename=LOCAL_FILE_PATH,
)

file_size_mb = os.path.getsize(LOCAL_FILE_PATH) / (1024 ** 2)
print(f"✅ Download complete — {file_size_mb:.2f} MB")

# =========================
# DESTINATION: RUNPOD S2 (EU-RO-1)
# =========================

DEST_BUCKET = "e9tcw5eupu"

# Store in a subfolder for clarity (optional)
DEST_KEY = "efficientnet/eff_ann_version8.h5"

print("🔐 Creating RunPod S2 client (EU-RO-1)")

runpod_s3 = boto3.client(
    "s3",
    aws_access_key_id="user_37sKcYrvnk9UXaIY3B3Zr90MH0g",
    aws_secret_access_key="rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n",
    endpoint_url="https://s3api-eu-ro-1.runpod.io",
    region_name="eu-ro-1",
)

print("✅ RunPod S2 client created")

# =========================
# MULTIPART CONFIG
# =========================

# Still safe to use multipart even for smaller files
transfer_config = TransferConfig(
    multipart_threshold=50 * 1024 * 1024,   # 50 MB
    multipart_chunksize=50 * 1024 * 1024,    # 50 MB
    max_concurrency=2,
    use_threads=True,
)

# =========================
# UPLOAD
# =========================

print("⬆️ Uploading EfficientNet model to RunPod S2")
print(f"Destination: s3://{DEST_BUCKET}/{DEST_KEY}")

runpod_s3.upload_file(
    Filename=LOCAL_FILE_PATH,
    Bucket=DEST_BUCKET,
    Key=DEST_KEY,
    Config=transfer_config,
)

print("🎉 Upload completed successfully!")
print(f"📍 s3://{DEST_BUCKET}/{DEST_KEY}")


🚀 Starting AWS S3 → RunPod S2 transfer (EfficientNet model)
📥 Downloading EfficientNet model from AWS S3
✅ Download complete — 15.08 MB
🔐 Creating RunPod S2 client (EU-RO-1)
✅ RunPod S2 client created
⬆️ Uploading EfficientNet model to RunPod S2
Destination: s3://e9tcw5eupu/efficientnet/eff_ann_version8.h5
🎉 Upload completed successfully!
📍 s3://e9tcw5eupu/efficientnet/eff_ann_version8.h5
